In [109]:
from dotenv import load_dotenv

load_dotenv()

True

In [110]:
from langchain_teddynote import logging
logging.langsmith("CLASS")

LangSmith 추적을 시작합니다.
[프로젝트명]
CLASS


In [111]:
from langchain_community.document_loaders import PDFPlumberLoader

In [112]:
# pip install pdfplumber

In [160]:
loader = PDFPlumberLoader('./data/converted_data_with_metadata.pdf')

In [161]:
docs = loader.load()

In [162]:
len(docs)

349

In [163]:
docs

[Document(metadata={'source': './data/converted_data_with_metadata.pdf', 'file_path': './data/converted_data_with_metadata.pdf', 'page': 0, 'total_pages': 349, 'Producer': 'PyPDF2', 'Title': 'Jump to Phython', 'Author': 'Park', 'Subject': 'Python', 'Keywords': 'Python, Book', 'Creator': 'PyPDF2 Library'}, page_content="1장\n01장 파이썬이란 무엇인가?\n필자는 파이썬의 프롬프트(>>>)를 처음 본 순간부터 지금까지 줄곧 파이썬과 함께 지내\n온 듯하다. ‘프로그래밍은 어렵고 지루하다’라는 고정관념을 가지고 있던 필자에게 파이\n썬은 커다란 충격으로 다가왔다. 여러분도 이 책을 통해 파이썬의 매력에 흠뻑 빠져 보\n기를 바란다. 01장에서는 파이썬의 특징과 장단점을 알아보고 파이썬 프로그래밍을 위한\n환경 구축 방법에 대해 배운다. 그리고 간단한 파이썬 프로그램도 작성해 본다.\n01-1 파이썬이란?\n파이썬(Python)은 1990년 암스테르담의 귀도 반 로섬(Guido van rossum)이 개발한 인터\n프리터 언어이다. 귀도는 파이썬이라는 이름을 자신이 좋아하는 코미디 쇼인 ‘몬티 파이\n썬의 날아다니는 서커스(Monty python's flying circus)’에서 따왔다고 한다.\n인터프리터 언어란 소스 코드를 한 줄씩 해석한 후 그때그때 실행해 결과를 바로 확인할\n수 있는 언어를 말한다.\n파이썬의 사전적 의미는 ‘고대 신화에 나오는 파르나소스 산의 동굴에 살던 큰 뱀’을 뜻\n하며, 아폴로 신이 델파이에서 파이썬을 퇴치했다는 이야기가 전해지고 있다. 대부분의\n파이썬 책 표지와 아이콘이 뱀 모양으로 그려져 있는 이유는 바로 이 때문이다.\n파이썬은 컴퓨터 프로그래밍을 교육할 때뿐만 아니라 기업

In [164]:
for doc in docs:
    doc.metadata['filename'] = doc.metadata['source']

In [165]:
# !pip install -q ragas==0.1.19

In [166]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [167]:
# 생성기
generator_llm = ChatOpenAI(model = 'gpt-4o-mini')

In [168]:
# 평가를 할 수 있는 비평기
critic_llm = ChatOpenAI(model = 'gpt-4o-mini')

In [169]:
# 임베딩 모델
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [170]:
# 텍스트 분할기
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

In [171]:
langchain_llm = LangchainLLMWrapper(ChatOpenAI(model = 'gpt-4o-mini'))
# 구문추출기 생성을 위한 모델 호출(RAGAS와의 호환을 위한 Wrapper)

In [172]:
# 구문추출기 : 문서 핵심 정보 식별 및 추출 역할
Keyphrase_extractor = KeyphraseExtractor(llm = langchain_llm)

In [173]:
# 임베딩 모델도 RAGAS와 호환을 위해 Wrapper 적용
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

In [174]:
# 저장소, docstore 만들기
docstore = InMemoryDocumentStore(
    splitter = splitter,
    embeddings = ragas_embeddings,
    extractor = Keyphrase_extractor
)

In [175]:
# 생성기 만들기
generator = TestsetGenerator.from_langchain(
    generator_llm, # 생성기
    critic_llm,  # 판별기
    ragas_embeddings, # 임베딩모델
    docstore = docstore # 문서저장소
)

In [176]:
# # 질문 유형 분포를 설정하는 딕셔너리입니다.
# # 각 질문 유형과 그에 해당하는 비율(0~1)을 정의합니다.
# distributions = {
#     "simple": 0.4,          # 단순한 질문 유형의 비율 (전체의 40%)
#     "reasoning": 0.2,       # 논리적 사고가 필요한 질문 유형의 비율 (전체의 20%)
#     "multi_context": 0.2,   # 여러 문맥을 고려해야 하는 질문 유형의 비율 (전체의 20%)
#     "conditional": 0.2      # 조건에 따라 답변이 달라지는 질문 유형의 비율 (전체의 20%)
# }

In [177]:
# 질문 유형 분포 결정
distributions = {simple: 0.2, reasoning: 0.3, multi_context: 0.3, conditional: 0.2}
# distributions = {simple : 0.4, reasoning : 0.2, multi_context : 0.2, conditional : 0.2}

In [179]:
def batch_documents(documents, batch_size):
    """
    문서를 일정 크기의 배치로 나눕니다.
    
    Args:
        documents (list): 문서 목록.
        batch_size (int): 각 배치의 크기.
    
    Returns:
        list: 배치로 나뉜 문서 그룹.
    """
    for i in range(0, len(documents), batch_size):
        yield documents[i:i + batch_size]

def generate_testset_in_batches(generator, documents, test_size, distributions, batch_size=10):
    """
    문서를 배치 단위로 나누어 처리하여 테스트 세트를 생성합니다.
    
    Args:
        generator: 테스트 세트 생성기 객체.
        documents (list): 전체 문서 목록.
        test_size (int): 각 배치에서 생성할 질문 세트 크기.
        distributions (dict): 질문 유형 분포.
        batch_size (int): 배치 크기 (기본값: 10).
    
    Returns:
        list: 각 배치의 결과를 결합한 전체 테스트 세트.
    """
    all_testsets = []
    for batch_index, document_batch in enumerate(batch_documents(documents, batch_size)):
        print(f"Processing batch {batch_index + 1}/{len(documents) // batch_size + 1}...")
        try:
            batch_testset = generator.generate_with_langchain_docs(
                documents=document_batch,
                test_size=test_size,
                distributions=distributions,
                with_debugging_logs=True,
                raise_exceptions=False
            )
            all_testsets.extend(batch_testset.to_dict())  # 배치 결과를 결합
        except Exception as e:
            print(f"Error processing batch {batch_index + 1}: {e}")
    
    return all_testsets

# 예제 사용
testset = generate_testset_in_batches(
    generator=generator,
    documents=docs,  # 전체 문서
    test_size=10,    # 각 배치에서 생성할 테스트 세트 크기
    distributions=distributions,
    batch_size=50    # 배치 크기
)

# 결과를 pandas DataFrame으로 변환
import pandas as pd
test_df = pd.DataFrame(testset)

# DataFrame 확인
print(test_df.head())


Processing batch 1/7...


KeyboardInterrupt: 

In [ ]:
# testset = generator.generate_with_langchain_docs(
#     documents = docs,
#     test_size = 10,
#     distributions = distributions,
#     with_debugging_logs = True,
#     # 로그 활성화
#     raise_exceptions= False
#     # 예외가 발생해도 계속 진행, 예외 발생 시 문서가 처리 되지 않거나 문서에 기록됨
# )

embedding nodes:  23%|██▎       | 12/52 [00:01<00:02, 14.22it/s][ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Division and multiplication', 'Variable assignment', 'String output', 'Python programming', 'Syntax error']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['GUI programming', 'Tkinter', 'Python as glue language', 'Raspberry Pi', 'Internet of Things']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Python programming', 'Conditional statement', 'If statement example', 'Indentation rules', 'For loop example']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['파일 관리', '로그 분석', '시스템 모니터링', '유틸리티 프로그램', 'GUI 프로그래밍']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['Python as a human language', 'Easy and fast to learn syntax', 'Free yet powerful', 'Programming enjoyment', 'Rapid development speed']}
embedding nodes:  37%|███▋      | 19/52 [00:01<00:01, 25.10it/s][ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['IDLE shell', 'Python p

In [ ]:
test_df = testset.to_pandas()
test_df

AttributeError: 'list' object has no attribute 'to_pandas'

In [ ]:
df = test_df

In [ ]:
df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How is Python described as a human language in...,[• 파이썬은 인간다운 언어이다\n• 문법이 쉬워 빠르게 배울 수 있다\n• 무료이...,Python is described as a human language with v...,simple,[{'source': './data/converted_data_with_metada...,True
1,What are some advantages of using Python progr...,[무료이지만 강력하다\n파이썬은 오픈 소스(open source)이며 무료 소프트웨...,"Python is open source and free software, allow...",simple,[{'source': './data/converted_data_with_metada...,True
2,"In the context of programming loops, what Engl...",[3\nfor 문을 사용하면 실행해야 할 문장을 여러 번 반복해서 실행할 수 있다....,The English term associated with the concept o...,simple,[{'source': './data/converted_data_with_metada...,True
3,How can the Python homepage be used to assist ...,[서의 설치 방법만 다룬다. 다른 시스템을 사용한다면 파이썬 홈페이지(www.pyt...,The Python homepage (www.python.org) can be us...,simple,[{'source': './data/converted_data_with_metada...,True
4,Which library helps with data analysis & viz u...,"[쉽게 할 수 있도록 사이킷런(scikit-learn), 텐서플로(TensorFlo...",The library that helps with data analysis and ...,reasoning,[{'source': './data/converted_data_with_metada...,True


In [ ]:
from datasets import Dataset

test_dataset = Dataset.from_pandas(df)
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done'],
    num_rows: 10
})

In [ ]:
df['contexts'][0]

['• 파이썬은 인간다운 언어이다\n• 문법이 쉬워 빠르게 배울 수 있다\n• 무료이지만 강력하다\n• 간결하다\n• 프로그래밍을 즐기게 해 준다\n• 개발 속도가 빠르다\n파이썬은 인간다운 언어이다\n프로그래밍이란 인간이 생각하는 것을 컴퓨터에 지시하는 행위를 말한다. 파이썬은 사람\n이 생각하는 방식을 그대로 표현할 수 있는 언어이다. 따라서 파이썬을 사용하는 프로그\n래머는 굳이 컴퓨터의 사고 체계에 맞추어 프로그래밍하려고 애쓸 필요가 없다. 이제 곧\n어떤 프로그램을 구상하자마자 머릿속에서 생각한 대로 코드를 술술 써 내려가는 자신의\n모습을 보고 놀라게 될 것이다.\n다음 소스 코드를 보면 이 말이 쉽게 이해될 것이다.\nif 4in [1,2,3,4]: print("4가 있습니다")\n이 소스 코드는 다음처럼 읽을 수 있다.\n만약 4가 1, 2, 3, 4 중에 있으면 "4가 있습니다"를 출력한다.\n프로그램을 모르더라도 직관적으로 무엇을 뜻하는지 알 수 있지 않는가? 마치 영어 문장\n을 읽는 듯한 착각에 빠져든다.\n문법이 쉬워 빠르게 배울 수 있다\n어려운 문법과 수많은 규칙에 둘러싸인 언어에서 벗어나고 싶지 않은가? 파이썬은 문법\n이 매우 쉽고 간결하며 사람의 사고 체계와 매우 닮아 있다. 배우기 쉬운 언어, 활용하기\n쉬운 언어가 가장 좋은 언어가 아닐까? 유명한 프로그래머인 에릭 레이먼드(Eric\nraymond)는 파이썬을 공부한 지 단 하루 만에 자신이 원하는 프로그램을 작성할 수 있\n었다고 한다.\n프로그래밍 경험이 조금이라도 있다면 파이썬의 자료형, 함수, 클래스 만드는 법, 라이브\n러리 및 내장 함수 사용 방법 등을 익히는 데 일주일이면 충분하다고 생각한다.']

In [ ]:
import ast
def convert_to_list(example):
    contexts = ast.literal_eval(example['contexts'])
    return {'contexts':contexts}

In [136]:
test_dataset = test_dataset.map(convert_to_list)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]


ValueError: malformed node or string: ['• 파이썬은 인간다운 언어이다\n• 문법이 쉬워 빠르게 배울 수 있다\n• 무료이지만 강력하다\n• 간결하다\n• 프로그래밍을 즐기게 해 준다\n• 개발 속도가 빠르다\n파이썬은 인간다운 언어이다\n프로그래밍이란 인간이 생각하는 것을 컴퓨터에 지시하는 행위를 말한다. 파이썬은 사람\n이 생각하는 방식을 그대로 표현할 수 있는 언어이다. 따라서 파이썬을 사용하는 프로그\n래머는 굳이 컴퓨터의 사고 체계에 맞추어 프로그래밍하려고 애쓸 필요가 없다. 이제 곧\n어떤 프로그램을 구상하자마자 머릿속에서 생각한 대로 코드를 술술 써 내려가는 자신의\n모습을 보고 놀라게 될 것이다.\n다음 소스 코드를 보면 이 말이 쉽게 이해될 것이다.\nif 4in [1,2,3,4]: print("4가 있습니다")\n이 소스 코드는 다음처럼 읽을 수 있다.\n만약 4가 1, 2, 3, 4 중에 있으면 "4가 있습니다"를 출력한다.\n프로그램을 모르더라도 직관적으로 무엇을 뜻하는지 알 수 있지 않는가? 마치 영어 문장\n을 읽는 듯한 착각에 빠져든다.\n문법이 쉬워 빠르게 배울 수 있다\n어려운 문법과 수많은 규칙에 둘러싸인 언어에서 벗어나고 싶지 않은가? 파이썬은 문법\n이 매우 쉽고 간결하며 사람의 사고 체계와 매우 닮아 있다. 배우기 쉬운 언어, 활용하기\n쉬운 언어가 가장 좋은 언어가 아닐까? 유명한 프로그래머인 에릭 레이먼드(Eric\nraymond)는 파이썬을 공부한 지 단 하루 만에 자신이 원하는 프로그램을 작성할 수 있\n었다고 한다.\n프로그래밍 경험이 조금이라도 있다면 파이썬의 자료형, 함수, 클래스 만드는 법, 라이브\n러리 및 내장 함수 사용 방법 등을 익히는 데 일주일이면 충분하다고 생각한다.']

In [ ]:
test_dataset[0]['contexts']

['• 파이썬은 인간다운 언어이다\n• 문법이 쉬워 빠르게 배울 수 있다\n• 무료이지만 강력하다\n• 간결하다\n• 프로그래밍을 즐기게 해 준다\n• 개발 속도가 빠르다\n파이썬은 인간다운 언어이다\n프로그래밍이란 인간이 생각하는 것을 컴퓨터에 지시하는 행위를 말한다. 파이썬은 사람\n이 생각하는 방식을 그대로 표현할 수 있는 언어이다. 따라서 파이썬을 사용하는 프로그\n래머는 굳이 컴퓨터의 사고 체계에 맞추어 프로그래밍하려고 애쓸 필요가 없다. 이제 곧\n어떤 프로그램을 구상하자마자 머릿속에서 생각한 대로 코드를 술술 써 내려가는 자신의\n모습을 보고 놀라게 될 것이다.\n다음 소스 코드를 보면 이 말이 쉽게 이해될 것이다.\nif 4in [1,2,3,4]: print("4가 있습니다")\n이 소스 코드는 다음처럼 읽을 수 있다.\n만약 4가 1, 2, 3, 4 중에 있으면 "4가 있습니다"를 출력한다.\n프로그램을 모르더라도 직관적으로 무엇을 뜻하는지 알 수 있지 않는가? 마치 영어 문장\n을 읽는 듯한 착각에 빠져든다.\n문법이 쉬워 빠르게 배울 수 있다\n어려운 문법과 수많은 규칙에 둘러싸인 언어에서 벗어나고 싶지 않은가? 파이썬은 문법\n이 매우 쉽고 간결하며 사람의 사고 체계와 매우 닮아 있다. 배우기 쉬운 언어, 활용하기\n쉬운 언어가 가장 좋은 언어가 아닐까? 유명한 프로그래머인 에릭 레이먼드(Eric\nraymond)는 파이썬을 공부한 지 단 하루 만에 자신이 원하는 프로그램을 작성할 수 있\n었다고 한다.\n프로그래밍 경험이 조금이라도 있다면 파이썬의 자료형, 함수, 클래스 만드는 법, 라이브\n러리 및 내장 함수 사용 방법 등을 익히는 데 일주일이면 충분하다고 생각한다.']

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
loader = PyMuPDFLoader('data/converted_data_with_metadata.pdf')
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
split_documents = text_splitter.split_documents(docs)

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
prompt = PromptTemplate.from_template(
    """당신은 질문에 따라 답변을 수행하는 친절한 AI 용어집과 퀴즈를 제공하는 학습 비서입니다. 
    당신은 주어진 context에서 주어진 question에 답하는 것을 수행합니다. 검색된 결과인 다음 context를 사용하여 질문인 question에 답하세요.
    만약, context에서 답을 찾을 수 없거나, 답을 모른다면 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.'라고 한글로 답변해 주세요.
    이름이나 기술적인 용어는 번역하지 않고 그대로 출력해주세요.
    
    
    # Context : {context}
    # Question : {question}
    # Answer :
    """
)

In [ ]:
llm = ChatOpenAI(model = 'gpt-4o-mini', temperature = 0)

In [ ]:
chain = (
    {'context' : retriever, 'question' : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
batch_dataset = []
for question in test_dataset['question']:
    batch_dataset.append(question)

In [ ]:
batch_dataset[:3]

['How is Python described as a human language in terms of its syntax and learning curve?',
 'What are some advantages of using Python programming in open source software development?',
 'In the context of programming loops, what English term is associated with the concept of repeating actions until a condition is satisfied?']

In [ ]:
answer = chain.batch(batch_dataset)
answer[:3]

['파이썬은 문법이 매우 쉽고 간결하며 사람의 사고 체계와 매우 닮아 있다고 설명됩니다. 프로그래머는 컴퓨터의 사고 체계에 맞추어 프로그래밍하려고 애쓸 필요가 없으며, 생각한 대로 코드를 쉽게 작성할 수 있습니다. 또한, 파이썬은 배우기 쉬운 언어로, 프로그래밍 경험이 조금이라도 있다면 자료형, 함수, 클래스 만드는 법, 라이브러리 및 내장 함수 사용 방법 등을 익히는 데 일주일이면 충분하다고 합니다. 이러한 점에서 파이썬은 빠르게 배울 수 있는 언어로 인식됩니다.',
 '주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.',
 'The English term associated with the concept of repeating actions until a condition is satisfied in programming loops is "while loop."']

In [ ]:
if 'answer' in test_dataset.column_names:
    test_dataset = test_dataset.remove_columns(['answer']).add_column('answer', answer)
else:
    test_dataset = test_dataset.add_column('answer', answer)

In [ ]:
test_dataset

Dataset({
    features: ['question', 'contexts', 'ground_truth', 'evolution_type', 'metadata', 'episode_done', 'answer'],
    num_rows: 10
})

In [ ]:
from ragas import evaluate
from ragas.metrics import answer_relevancy,faithfulness,context_recall,context_precision

In [ ]:
result = evaluate(
    dataset = test_dataset,
    metrics = [
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision
    ]
)


# answer_relevancy : 모델이 문맥에서 중요한 정보를 잘 회상, 재현했는가
# faithfulness : 모델의 답변이 문맥에 기반해서 사실적인가(factual)
# context_recall : 전체 질문에 대해 모델의 답변이 질문과 얼마나 관련성이 있는가
# context_precision : 모델이 문맥에서 필요한 정보를 정확히 활용했는가

Evaluating: 100%|██████████| 40/40 [00:13<00:00,  2.86it/s]


In [ ]:
result

{'answer_relevancy': 0.2763, 'faithfulness': 0.4022, 'context_recall': 0.8333, 'context_precision': 0.6000}

In [ ]:
result_df = result.to_pandas()
result_df

,question,contexts,answer,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,How is Python described as a human language in...,[• 파이썬은 인간다운 언어이다\n• 문법이 쉬워 빠르게 배울 수 있다\n• 무료이...,파이썬은 문법이 매우 쉽고 간결하며 사람의 사고 체계와 매우 닮아 있다고 설명됩니다...,Python is described as a human language with v...,0.000000,0.555556,0.000000,1.0
1,What are some advantages of using Python progr...,[무료이지만 강력하다\n파이썬은 오픈 소스(open source)이며 무료 소프트웨...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,"Python is open source and free software, allow...",0.000000,0.000000,0.333333,0.0
2,"In the context of programming loops, what Engl...",[3\nfor 문을 사용하면 실행해야 할 문장을 여러 번 반복해서 실행할 수 있다....,The English term associated with the concept o...,The English term associated with the concept o...,0.977379,1.000000,1.000000,1.0
3,How can the Python homepage be used to assist ...,[서의 설치 방법만 다룬다. 다른 시스템을 사용한다면 파이썬 홈페이지(www.pyt...,파이썬 홈페이지(www.python.org)를 사용하여 다양한 시스템에서 파이썬 설...,The Python homepage (www.python.org) can be us...,0.897953,0.666667,1.000000,1.0
4,Which library helps with data analysis & viz u...,"[쉽게 할 수 있도록 사이킷런(scikit-learn), 텐서플로(TensorFlo...",파이썬에서 데이터 분석 및 시각화를 도와주는 라이브러리는 넘파이(NumPy)와 판다...,The library that helps with data analysis and ...,0.887899,1.000000,1.000000,1.0
5,How do inputs lead to a function's result?,"[>>> add(3,4)\n7\ndef는 파이썬에서 함수를 정의할 때 사용하는 예약...",입력값은 함수가 수행할 작업의 기초가 됩니다. 함수는 입력값을 받아 특정 작업을 수...,"In the context of the add function, inputs a a...",0.000000,0.800000,1.000000,1.0
6,Which iterative method allows sequential acces...,[>>> a\n'Python'\n조건문 if\n다음은 간단한 조건문 if를 사용한 ...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The answer to given question is not present in...,0.000000,0.000000,1.000000,0.0
7,How do inline annotations in .py files aid fut...,[그러면 다음과 같은 IDLE 셸(shell) 창이 나타난다.\nIDLE 셸은 ID...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The answer to given question is not present in...,0.000000,0.000000,1.000000,0.0
8,How does saving a program affect its support i...,[그러면 다음과 같은 IDLE 셸(shell) 창이 나타난다.\nIDLE 셸은 ID...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,The answer to given question is not present in...,0.000000,0.000000,1.000000,0.0
9,How do comments work in a Python script run in...,[그러면 다음과 같은 IDLE 셸(shell) 창이 나타난다.\nIDLE 셸은 ID...,주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.,Comments in a Python script run in IDLE are in...,0.000000,0.000000,1.000000,1.0


In [ ]:
result_df.loc[:, 'context_recall':'context_precision']

,context_recall,context_precision
0,0.000000,1.0
1,0.333333,0.0
2,1.000000,1.0
3,1.000000,1.0
4,1.000000,1.0
5,1.000000,1.0
6,1.000000,0.0
7,1.000000,0.0
8,1.000000,0.0
9,1.000000,1.0


|context_precision (문맥 정밀도)|faithfulness (신뢰성)|answer_relevancy (답변 적합성)|context_recall (문맥 재현율)|
|---|---|---|---|
|답변에 사용된 문맥이 정확했는지|답변이 문맥에 기반해 사실적(factual)인지|답변이 질문과 관련성이 있는지|답변이 문맥에서 얼마나 많은 정보를 활용했는지|
|1.0: 답변에 사용된 정보가 문맥에서 매우 정확하게 활용됨|1.0: 답변이 문맥과 매우 정확하게 일치|1.0: 답변이 질문과 매우 높은 관련성을 가짐|1.0: 답변이 문맥의 모든 중요한 정보를 잘 회상|
|0.0: 답변이 문맥과 무관하거나, 부정확한 정보를 활용|0.0: 답변이 문맥에 기반하지 않고, 사실과 다를 가능성이 큼|0.0: 답변이 질문과 관련이 없거나, 부정확|0.0: 답변이 문맥을 활용하지 못함|